## Part 1

In [1]:
pip install rapidfuzz

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import re
from rapidfuzz import process, fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

resolved_df = pd.read_csv('resolved_queries.csv')
unresolved_df = pd.read_csv('new_queries.csv')

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

resolved_df['clean'] = resolved_df['Pre_Resolved_Query'].apply(preprocess)
unresolved_df['clean'] = unresolved_df['Variation_Query'].apply(preprocess)

# --- 1. Fuzzy Matching with RapidFuzz ---

def fuzzy_match(unresolved_texts, resolved_texts, scorer=fuzz.token_set_ratio, threshold=80):
    matches = []
    for text in unresolved_texts:
        best_match = process.extractOne(text, resolved_texts, scorer=scorer)
        if best_match and best_match[1] >= threshold:
            matches.append((text, best_match[0], best_match[1]))
        else:
            matches.append((text, None, None))
    return matches

fuzzy_results = fuzzy_match(unresolved_df['clean'], resolved_df['clean'], threshold=80)

print("Fuzzy Matching Results (threshold=80):")
for original, matched, score in fuzzy_results:
    print(f"Query: '{original}'\nMatched resolved: '{matched}'\nScore: {score}\n")

# --- 2. TF-IDF + Cosine Similarity ---

vectorizer = TfidfVectorizer()

# Vectorize resolved queries
tfidf_resolved = vectorizer.fit_transform(resolved_df['clean'])
# Vectorize unresolved queries
tfidf_unresolved = vectorizer.transform(unresolved_df['clean'])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_unresolved, tfidf_resolved)

# For each unresolved query, find best match and score
tfidf_matches = []
threshold_cosine = 0.7

for i, sims in enumerate(cosine_sim):
    max_idx = sims.argmax()
    max_score = sims[max_idx]
    if max_score >= threshold_cosine:
        tfidf_matches.append((unresolved_df['clean'].iloc[i], resolved_df['clean'].iloc[max_idx], max_score))
    else:
        tfidf_matches.append((unresolved_df['clean'].iloc[i], None, None))

print("\nTF-IDF Cosine Similarity Results (threshold=0.7):")
for original, matched, score in tfidf_matches:
    print(f"Query: '{original}'\nMatched resolved: '{matched}'\nScore: {score}\n")


Fuzzy Matching Results (threshold=80):
Query: 'unabel to conect to the internet'
Matched resolved: 'unable to connect to the internet'
Score: 94.91525423728814

Query: 'cant connect to internet'
Matched resolved: 'unable to connect to the internet'
Score: 88.37209302325581

Query: 'intenet not working'
Matched resolved: 'None'
Score: None

Query: 'payment failed while chekout'
Matched resolved: 'payment failed during checkout'
Score: 82.75862068965517

Query: 'payment did not go through during chckout'
Matched resolved: 'None'
Score: None

Query: 'payment issue at check out'
Matched resolved: 'None'
Score: None

Query: 'application crashes when opening setings'
Matched resolved: 'app crashes when opening settings'
Score: 87.67123287671232

Query: 'app crash when going to settings'
Matched resolved: 'app crashes when opening settings'
Score: 86.15384615384616

Query: 'settings cause the app to chrash'
Matched resolved: 'None'
Score: None

Query: 'forgot passwrd and cant reset'
Matched r

In [4]:
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('all-MiniLM-L6-v2')

resolved_sentences = resolved_df['Pre_Resolved_Query'].tolist()
unresolved_sentences = unresolved_df['Variation_Query'].tolist()

encoded_resolved = model.encode(resolved_sentences, convert_to_tensor=True)
encoded_unresolved = model.encode(unresolved_sentences, convert_to_tensor=True)

cosine_scores = util.pytorch_cos_sim(encoded_unresolved, encoded_resolved)

top_matches = []
for i in range(len(unresolved_sentences)):
    scores = cosine_scores[i]
    max_score, idx = torch.max(scores, dim=0)
    if max_score > 0.7:  # example threshold
        top_matches.append((unresolved_sentences[i], resolved_sentences[idx], max_score.item()))
    else:
        top_matches.append((unresolved_sentences[i], None, None))

for um, rm, score in top_matches:
    print(f"Unresolved: {um}\nMatched: {rm}\nScore: {score}\n")



Unresolved: Unabel to conect to the internet
Matched: None
Score: None

Unresolved: Can’t connect to internet
Matched: Unable to connect to the internet
Score: 0.9712159633636475

Unresolved: Intenet not working
Matched: None
Score: None

Unresolved: Payment failed while chekout
Matched: Payment failed during checkout
Score: 0.8156481981277466

Unresolved: Payment did not go through during chckout
Matched: None
Score: None

Unresolved: Payment issue at check out
Matched: Payment failed during checkout
Score: 0.7518054842948914

Unresolved: Application crashes when opening setings
Matched: App crashes when opening settings
Score: 0.8146043419837952

Unresolved: App crash when going to settings
Matched: App crashes when opening settings
Score: 0.9565289616584778

Unresolved: Settings cause the app to chrash
Matched: None
Score: None

Unresolved: Forgot passwrd and cant reset
Matched: Forgot password and unable to reset
Score: 0.7264114618301392

Unresolved: Forgotten password, unable to

In [5]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
import torch

resolved_df = pd.read_csv('resolved_queries.csv')
unresolved_df = pd.read_csv('new_queries.csv')

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

resolved_df['clean'] = resolved_df['Pre_Resolved_Query'].apply(preprocess)
unresolved_df['clean'] = unresolved_df['Variation_Query'].apply(preprocess)

#Load model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

#Encode resolved and unresolved queries
encoded_resolved = model.encode(resolved_df['clean'].tolist(), convert_to_tensor=True)
encoded_unresolved = model.encode(unresolved_df['clean'].tolist(), convert_to_tensor=True)

#Compute cosine similarity matrix
cosine_scores = util.pytorch_cos_sim(encoded_unresolved, encoded_resolved)  # Shape: (num_unresolved, num_resolved)

#Find closest match for each unresolved query
closest_matches = []
for i in range(cosine_scores.size(0)):
    scores = cosine_scores[i]
    max_score, idx = torch.max(scores, dim=0)
    idx = int(idx)  # <-- convert tensor scalar to int here
    best_resolved = resolved_df['Pre_Resolved_Query'].iloc[idx]
    original_unresolved = unresolved_df['Variation_Query'].iloc[i]
    closest_matches.append((original_unresolved, best_resolved, max_score.item()))


for unresolved, resolved, score in closest_matches:
    print(f"Unresolved: {unresolved}")
    print(f"Closest resolved match: {resolved}")
    print(f"Similarity score: {score:.4f}")
    print("-" * 50)


Unresolved: Unabel to conect to the internet
Closest resolved match: Unable to connect to the internet
Similarity score: 0.3449
--------------------------------------------------
Unresolved: Can’t connect to internet
Closest resolved match: Unable to connect to the internet
Similarity score: 0.9687
--------------------------------------------------
Unresolved: Intenet not working
Closest resolved match: App crashes when opening settings
Similarity score: 0.1856
--------------------------------------------------
Unresolved: Payment failed while chekout
Closest resolved match: Payment failed during checkout
Similarity score: 0.8156
--------------------------------------------------
Unresolved: Payment did not go through during chckout
Closest resolved match: Payment failed during checkout
Similarity score: 0.6416
--------------------------------------------------
Unresolved: Payment issue at check out
Closest resolved match: Payment failed during checkout
Similarity score: 0.7518
-------

In [6]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

variations_df = pd.read_csv('name_variations.csv')
base_names_df = pd.read_csv('base_names.csv')

def preprocess_name(name):
    name = str(name).lower().strip()
    # Handle "last, first" format -> "first last"
    if ',' in name:
        parts = [part.strip() for part in name.split(',')]
        if len(parts) == 2:
            name = parts[1] + ' ' + parts[0]
    # Remove multiple spaces
    name = re.sub(r'\s+', ' ', name)
    return name

variations_df['clean'] = variations_df['Variation'].apply(preprocess_name)
base_names_df['clean'] = base_names_df['Base_Name'].apply(preprocess_name)

base_name_list = base_names_df['clean'].tolist()

# Fuzzy match each variation against base names using token_set_ratio
def get_best_match(name, choices, scorer=fuzz.token_set_ratio, threshold=75):
    match = process.extractOne(name, choices, scorer=scorer)
    if match and match[1] >= threshold:
        return match[0], match[1]
    else:
        return None, None

results = []

for idx, row in variations_df.iterrows():
    variation_name = row['clean']
    best_match, score = get_best_match(variation_name, base_name_list)
    if best_match is None:
        best_match = 'No good match'
        score = 0
    results.append({
        'Original_Variation': row['Variation'],
        'Matched_Base_Name': base_names_df.loc[base_names_df['clean'] == best_match, 'Base_Name'].values[0] if best_match != 'No good match' else None,
        'Match_Score': score
    })

results_df = pd.DataFrame(results)
print(results_df)


   Original_Variation Matched_Base_Name  Match_Score
0        Thomas  King       Thomas King   100.000000
1          ThomasKing              None     0.000000
2        Maria Garcia      Maria Garcia   100.000000
3           MaryLewis              None     0.000000
4            Nancy W.      Nancy Wright    76.923077
..                ...               ...          ...
95    Jennifer- Brown    Jennifer Brown    96.551724
96      Daniel- Scott      Daniel Scott    96.000000
97           David M.    David Martinez    76.923077
98        Paul Allen.        Paul Allen    95.238095
99        Paul  Allen        Paul Allen   100.000000

[100 rows x 3 columns]


In [7]:
import re

def preprocess_name_advanced(name):
    name = str(name).strip()
    # Insert space before uppercase letters preceded by lowercase letters or digits
    name = re.sub(r'(?<=[a-z0-9])(?=[A-Z])', ' ', name)
    # Handle "last, first" to "first last"
    if ',' in name:
        parts = [part.strip() for part in name.split(',')]
        if len(parts) == 2:
            name = parts[1] + ' ' + parts[0]
    # Lowercase, remove punctuations except apostrophes for names like O'Connor
    name = name.lower()
    name = re.sub(r'[^a-z0-9\s\']', '', name)
    name = re.sub(r'\s+', ' ', name)
    return name.strip()

variations_df['clean'] = variations_df['Variation'].apply(preprocess_name_advanced)
base_names_df['clean'] = base_names_df['Base_Name'].apply(preprocess_name_advanced)

base_name_list = base_names_df['clean'].tolist()

# Fuzzy match each variation against base names using token_set_ratio
def get_best_match(name, choices, scorer=fuzz.token_set_ratio, threshold=75):
    match = process.extractOne(name, choices, scorer=scorer)
    if match and match[1] >= threshold:
        return match[0], match[1]
    else:
        return None, None

results = []

for idx, row in variations_df.iterrows():
    variation_name = row['clean']
    best_match, score = get_best_match(variation_name, base_name_list)
    if best_match is None:
        best_match = 'No good match'
        score = 0
    results.append({
        'Original_Variation': row['Variation'],
        'Matched_Base_Name': base_names_df.loc[base_names_df['clean'] == best_match, 'Base_Name'].values[0] if best_match != 'No good match' else None,
        'Match_Score': score
    })

results_df = pd.DataFrame(results)
print(results_df)


   Original_Variation Matched_Base_Name  Match_Score
0        Thomas  King       Thomas King   100.000000
1          ThomasKing       Thomas King   100.000000
2        Maria Garcia      Maria Garcia   100.000000
3           MaryLewis        Mary Lewis   100.000000
4            Nancy W.      Nancy Wright    83.333333
..                ...               ...          ...
95    Jennifer- Brown    Jennifer Brown   100.000000
96      Daniel- Scott      Daniel Scott   100.000000
97           David M.    David Martinez    83.333333
98        Paul Allen.        Paul Allen   100.000000
99        Paul  Allen        Paul Allen   100.000000

[100 rows x 3 columns]
